<a href="https://colab.research.google.com/github/jbell1991/jbell1991.github.io/blob/master/Robintrack_Final_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.style as style
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import chart_studio.tools as tls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [28]:
# loading in popularity data from robintrack.net, which lacks pricing data
# combined and cleaned in another environment due to size issues
df = pd.read_csv('/content/popularity_data.csv')
print(df.shape)
df.head()

(4428700, 3)


,users_holding,Symbol,Date
0,1524,OAS,2018-05-02
1,1517,OAS,2018-05-03
2,1522,OAS,2018-05-04
3,1520,OAS,2018-05-05
4,1520,OAS,2018-05-06


In [29]:
# loading in price data for S&P 500 stocks for the same period
# historical price data was scraped from yahoo finance
df2 = pd.read_csv('/content/combined_prices.csv')
print(df2.shape)
df2.head()

(210101, 4)


,Date,Close,Volume,Symbol
0,2018-05-02,43.860001,23601600.0,CSCO
1,2018-05-03,44.439999,23750200.0,CSCO
2,2018-05-04,45.299999,19504100.0,CSCO
3,2018-05-07,45.730000,17822400.0,CSCO
4,2018-05-08,45.709999,20285300.0,CSCO


In [30]:
# merging the popularity data with pricing data on symbol and date
users_prices = df2.merge(df, how ='inner', on = ('Symbol', 'Date'))
print(users_prices.shape)
users_prices.head()

(203927, 5)


,Date,Close,Volume,Symbol,users_holding
0,2018-05-02,43.860001,23601600.0,CSCO,23646
1,2018-05-03,44.439999,23750200.0,CSCO,23765
2,2018-05-04,45.299999,19504100.0,CSCO,23718
3,2018-05-07,45.730000,17822400.0,CSCO,23552
4,2018-05-08,45.709999,20285300.0,CSCO,23547


In [31]:
# checking for missing values
# although it says there are none here, there are some found later on
users_prices.isnull().sum()

Date             0
Close            0
Volume           0
Symbol           0
users_holding    0
dtype: int64

In [32]:
# checking datatypes of dataset
users_prices.dtypes

Date              object
Close            float64
Volume           float64
Symbol            object
users_holding      int64
dtype: object

In [43]:
# defining a funciton for a specified ticker that will display the ticker's
# head, a plotly graph reproducing the graph from Robintrack.net and 
# a correlation matrix showing correlation between price, users holding,
# and volume data points
def robinhood_data(ticker):
  ticker_data = users_prices[users_prices.values == ticker]
  ticker_data.reset_index(inplace=True)
  print(ticker_data.head())
  price_data = go.Scatter(x=ticker_data['Date'], y=ticker_data['Close'],
                          name='Price')
  users_data = go.Scatter(x=ticker_data['Date'], y=ticker_data['users_holding'],
                          yaxis='y2', name='Users Holding')
  layout = go.Layout(height=600, width=1000, 
                     title=ticker + ' Price v Users Holding',
                     xaxis=dict(title='Date'), yaxis=dict(title='Price'),
                     yaxis2=dict(title='Users Holding', showgrid=False, 
                                 overlaying='y', side='right'),
                                 legend_orientation='h')
  fig = go.Figure(data=[price_data, users_data], layout=layout)
  iplot(fig)
  print(ticker_data.corr())
# enter a ticker here to change graphs
robinhood_data('KHC')

   index        Date      Close      Volume Symbol  users_holding
0  45015  2018-05-02  54.200001   9045100.0    KHC           4242
1  45016  2018-05-03  54.950001  10380800.0    KHC           4306
2  45017  2018-05-04  58.009998   9221300.0    KHC           4353
3  45018  2018-05-07  58.830002   7337400.0    KHC           4387
4  45019  2018-05-08  58.759998   7163200.0    KHC           4476


                  index     Close    Volume  users_holding
index          1.000000 -0.920453  0.100665       0.881961
Close         -0.920453  1.000000 -0.203597      -0.962284
Volume         0.100665 -0.203597  1.000000       0.151283
users_holding  0.881961 -0.962284  0.151283       1.000000


In [34]:
# creating a correlation matrix for the whole dataset
users_prices.corr()

,Close,Volume,users_holding
Close,1.000000,-0.118196,0.055427
Volume,-0.118196,1.000000,0.626707
users_holding,0.055427,0.626707,1.000000


In [35]:
# pivoting the dataset to make it more readable, and easier to manipulate
# we can now see some of the missing values(NaNs)
users_prices_pivot = pd.pivot_table(users_prices, index=['Date'], columns=['Symbol'])
users_prices_pivot

Close                         ... users_holding               
Symbol              A        AAL         AAP  ...          ZBRA   ZION     ZTS
Date                                          ...                             
2018-05-02  65.910004  42.240002  115.900002  ...         195.0  246.0   588.0
2018-05-03  66.339996  42.110001  115.430000  ...         200.0  243.0   611.0
2018-05-04  67.000000  42.860001  116.669998  ...         201.0  244.0   609.0
2018-05-07  67.389999  42.439999  116.139999  ...         204.0  238.0   607.0
2018-05-08  67.370003  43.000000  116.800003  ...         225.0  237.0   604.0
...               ...        ...         ...  ...           ...    ...     ...
2019-12-24  85.300003  29.180000  160.570007  ...         778.0  243.0  1595.0
2019-12-26  85.449997  29.670000  159.270004  ...         776.0  245.0  1596.0
2019-12-27  85.419998  28.440001  158.350006  ...         780.0  246.0  1599.0
2019-12-30  84.900002  28.299999  159.119995  ...         784.0  246.0  1599.0
2019-12-31  85.309998  28.680000  160.160004  ...         782.0  245.0  1598.0

[415 rows x 1509 columns]

In [36]:
# dropping NaNs from the dataset
users_prices_pivot = users_prices_pivot.dropna(axis=1)
# dropping the volume column
users_prices_pivot = users_prices_pivot.drop(columns=['Volume'])
# replacing any zero values with a nonzero value to avoid any undefined errors
users_prices_pivot = users_prices_pivot.replace(0, value=.000001)
users_prices_pivot

Close                         ... users_holding               
Symbol              A        AAL         AAP  ...          ZBRA   ZION     ZTS
Date                                          ...                             
2018-05-02  65.910004  42.240002  115.900002  ...         195.0  246.0   588.0
2018-05-03  66.339996  42.110001  115.430000  ...         200.0  243.0   611.0
2018-05-04  67.000000  42.860001  116.669998  ...         201.0  244.0   609.0
2018-05-07  67.389999  42.439999  116.139999  ...         204.0  238.0   607.0
2018-05-08  67.370003  43.000000  116.800003  ...         225.0  237.0   604.0
...               ...        ...         ...  ...           ...    ...     ...
2019-12-24  85.300003  29.180000  160.570007  ...         778.0  243.0  1595.0
2019-12-26  85.449997  29.670000  159.270004  ...         776.0  245.0  1596.0
2019-12-27  85.419998  28.440001  158.350006  ...         780.0  246.0  1599.0
2019-12-30  84.900002  28.299999  159.119995  ...         784.0  246.0  1599.0
2019-12-31  85.309998  28.680000  160.160004  ...         782.0  245.0  1598.0

[415 rows x 962 columns]

In [0]:
# taking the first and last rows of the dataframe to calculate percentage change
first = users_prices_pivot.iloc[0]
last = users_prices_pivot.iloc[414]

In [38]:
# creating a new dataframe containing percentage change calculations
first_last = pd.DataFrame(((last / first) - 1) * 100)
first_last = first_last.T
first_last.stack().T

0              ...                      
Symbol                 A         AAL  ...      ZION         ZTS
Close          29.434066  -32.102275  ... -5.924987   65.644560
users_holding  96.610169  186.577896  ... -0.406504  171.768707

[2 rows x 481 columns]

In [0]:
# saved dataframe to a csv and deleted first row 0 using excel 
first_last.stack().T.to_csv('first_last.csv')

In [40]:
# reloaded the dataframe from excel
first_last_index = pd.read_csv('/content/first_last_index.csv', index_col=0)
first_last_index

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,...,UTX,V,VAR,VFC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,WAB,WAT,WBA,WCG,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WU,WY,WYNN,XEC,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Close,29.434066,-32.102275,38.188095,66.307969,-11.786392,-7.767416,-43.539423,50.146937,39.718664,49.167792,34.586628,4.321400,39.685403,-43.688835,46.463359,31.551904,36.594882,63.248567,17.425089,26.543979,-6.400435,46.229365,38.738351,20.558263,-25.347502,11.853132,6.878058,19.551350,65.788068,-5.330879,20.799530,318.049220,42.181034,42.282959,25.257534,68.403316,17.720803,-23.378287,60.270221,31.787245,...,26.379742,48.678587,22.169647,32.846653,-16.570155,26.684850,-4.054246,49.339996,63.218965,46.484241,10.253968,28.586391,-11.197349,23.500180,-7.368425,56.171958,-17.324471,43.258787,48.555858,3.084883,-4.350367,37.758379,40.969822,-9.292544,37.641883,40.200186,-26.724729,39.116887,-17.350848,-27.304614,-46.716069,36.420285,-9.140630,11.860051,14.387346,25.598513,27.430609,91.887028,-5.924987,65.644560
users_holding,96.610169,186.577896,-11.258278,42.541367,77.943939,-10.108303,339.583333,93.500838,50.933333,46.247034,6.372549,169.842932,55.586987,232.812500,83.421751,51.891892,89.683860,60.247486,66.512488,-11.596180,30.708661,54.545455,84.705882,5.898491,39.111709,17.900684,16.273850,-12.820513,71.153846,40.950792,-25.073407,31.495492,43.292683,13.484303,84.827586,85.432473,58.547958,23.430322,161.437908,160.497238,...,120.781250,135.390500,16.666667,76.806084,86.139283,89.605735,68.027211,97.500000,70.000000,50.842021,55.781818,89.075243,448.936170,-26.666667,217.914692,96.202532,37.474482,40.926641,62.491888,32.086168,76.380368,133.333333,177.127578,172.569444,21.667080,92.452830,369.421488,37.645108,267.671692,23.023170,243.362832,156.480000,72.728138,68.306011,49.319213,73.555909,26.060606,301.025641,-0.406504,171.768708


In [41]:
# exploring the dataset to see names of columns 
first_last_index_t = first_last_index.T
first_last_index_t = first_last_index_t.reset_index()
first_last_index_t

Symbol,index,Close,users_holding
0,A,29.434066,96.610169
1,AAL,-32.102275,186.577896
2,AAP,38.188095,-11.258278
3,AAPL,66.307969,42.541367
4,ABBV,-11.786392,77.943939
...,...,...,...
476,YUM,25.598513,73.555909
477,ZBH,27.430609,26.060606
478,ZBRA,91.887028,301.025641
479,ZION,-5.924987,-0.406504


In [42]:
# creating a plotly showing each stock's percent change in price and 
# users holding for the period 
fig = go.Figure(data=go.Scatter(x=first_last_index_t['users_holding'], 
                                y=first_last_index_t['Close'], mode='markers', 
                                text=first_last_index_t['index'],))

fig.update_layout(title='Percent Change in Price v Users Holding',
                  xaxis=dict(title='Percent Change in Users Holding'), 
                  yaxis=dict(title='Percent Change in Price'))
fig.show()